In [3]:
from django.core.management import setup_environ
import settings
setup_environ(settings)

'/home/wahern/projects/millstone/genome_designer'

In [3]:
import os
from main.models import ExperimentSampleToAlignment
sample_alignment = ExperimentSampleToAlignment.objects.get(experiment_sample__label='lib1_rec07')

genome_finish_dir = os.path.join(sample_alignment.get_model_data_dir(), 'genome_finishing')

In [6]:
genome_finish_dir

'/home/wahern/projects/millstone/genome_designer/conf/../temp_data/projects/6e766eb8/alignment_groups/9c54183d/sample_alignments/ca0c79af/genome_finishing'

In [7]:
read_unpacking_dir = os.path.join(genome_finish_dir, '0', 'read_unpacking')
#os.mkdir(read_unpacking_dir)

In [8]:
import subprocess
cmd = 'ls'

fn = os.path.join(read_unpacking_dir, 'test_output.txt')
with open(fn, 'w') as fh:
    subprocess.call(cmd, shell=True, stdout=fh)

In [15]:
velvet_dir = os.path.join(settings.TOOLS_DIR, 'velvet')
extract_contig_reads_executable = os.path.join(velvet_dir, 'contrib/extractContigReads/extractContigReads.pl')

assembly_dir = os.path.join(genome_finish_dir, '0', 'velvet_k21')

cmd = [extract_contig_reads_executable, '1', assembly_dir]
cmd = ' '.join(cmd)
print cmd

fn = os.path.join(read_unpacking_dir, 'extracted.fa')
with open(fn, 'w') as fh:
    subprocess.call(cmd, shell=True, stdout=fh)

/home/wahern/projects/millstone/genome_designer/conf/../tools/velvet/contrib/extractContigReads/extractContigReads.pl 1 /home/wahern/projects/millstone/genome_designer/conf/../temp_data/projects/6e766eb8/alignment_groups/9c54183d/sample_alignments/ca0c79af/genome_finishing/0/velvet_k21


In [62]:
import pysam
import re
p1 = re.compile('>(\S+)/(\d)')

# reads = []
# with open(fn) as fh:
#     counter = 0
#     for line in fh:
#         #print 'line', counter, ':'
#         print line
        
#         m1 = p1.match(line)
#         if m1:
#             reads.append({'read_id': m1.group(1), 'read_number': m1.group(2)})
#             print 're match:\n\tgroup(1):', m1.group(1), '\n\tgroup(2):', m1.group(2)
#         counter += 1
#         if counter == 4:
#             break

# print reads

sv_indicants_path = os.path.join(genome_finish_dir, '0/bwa_align.SV_indicants_with_pairs.bam')
sam_file = pysam.AlignmentFile(sv_indicants_path)
counter = 0
for read in sam_file:
    print 'read.query_name:', read.query_name
    counter += 1
    if counter == 5:
        break

read.query_name: HSQ700642:127:D0F9HACXX:5:1101:1126:71563
read.query_name: HSQ700642:127:D0F9HACXX:5:1101:1126:71563
read.query_name: HSQ700642:127:D0F9HACXX:5:1101:1126:95656
read.query_name: HSQ700642:127:D0F9HACXX:5:1101:1126:95656
read.query_name: HSQ700642:127:D0F9HACXX:5:1101:1127:6749


In [18]:
from main.models import Dataset
fq1_path = sample_alignment.experiment_sample.dataset_set.get(type=Dataset.TYPE.FASTQ1).wrap_if_compressed()
print fq1_path

<(gzip -dc /home/wahern/projects/millstone/genome_designer/conf/../temp_data/projects/6e766eb8/samples/eab3affa/tmprbj2gz_recoli_lib1_rec07_1.fq.gz)


In [39]:
import os
import re
import pysam
from collections import defaultdict
from main.models import ExperimentSampleToAlignment

velvet_dir = os.path.join(settings.TOOLS_DIR, 'velvet')
extract_contig_reads_executable = os.path.join(velvet_dir, 'contrib/extractContigReads/extractContigReads.pl')

sample_alignment = ExperimentSampleToAlignment.objects.get(experiment_sample__label='ins_1kb_sample')
genome_finish_dir = os.path.join(sample_alignment.get_model_data_dir(), 'genome_finishing')
# print 'genome_finish_dir:', genome_finish_dir
assembly_dir = os.path.join(genome_finish_dir, '1', 'velvet_k21')

read_unpacking_dir = os.path.join(genome_finish_dir, '1', 'read_unpacking')
if not os.path.exists(read_unpacking_dir):
    os.mkdir(read_unpacking_dir)

contig_number = 1
cmd = [extract_contig_reads_executable, str(contig_number), assembly_dir]
cmd = ' '.join(cmd)

contig_reads_fasta = os.path.join(read_unpacking_dir, 'contig_' + str(contig_number) + '_reads.fa')
if not os.path.exists(contig_reads_fasta):
    with open(contig_reads_fasta, 'w') as fh:
        subprocess.call(cmd, shell=True, stdout=fh)

p1 = re.compile('>(\S+)/(\d)')
contig_reads = defaultdict(list)
with open(contig_reads_fasta) as fh:
    for line in fh:
        m1 = p1.match(line)
        if m1:
            read_id = m1.group(1)
            read_number = int(m1.group(2))
            contig_reads[read_id].append(read_number)

# sv_indicants_path = os.path.join(genome_finish_dir, '1/bwa_align.SV_indicants_with_pairs.bam')
sv_indicants_path = os.path.join(genome_finish_dir, '1/bwa_align.clipped.bam')
sam_file = pysam.AlignmentFile(sv_indicants_path)
sv_indicant_reads_in_contig = []
# counter = 0
for read in sam_file:
    if read.is_read1:
        read_number = 1
    elif read.is_read2:
        read_number = 2
    else:
        raise Exception('Read is neither read1 nor read2')

    contig_read_numbers = contig_reads.get(read.query_name, [])
    if read_number in contig_read_numbers:
        sv_indicant_reads_in_contig.append(read)

# Verification
number_contig_reads = 0
for cr in contig_reads.values():
    number_contig_reads += len(cr)

# print 'number_contig_reads:', number_contig_reads
print 'len(sv_indicant_reads_in_contig):', len(sv_indicant_reads_in_contig)
# assert number_contig_reads == len(sv_indicant_reads_in_contig)

len(sv_indicant_reads_in_contig): 56


In [25]:
all_ref_pos = []
for read in sv_indicant_reads_in_contig:
    all_ref_pos.extend(read.get_reference_positions())

In [24]:
right_clip_positions = []
left_clip_positions = []

MATCH = 0
INS = 1
DEL = 2
SOFT_CLIP = 4
HARD_CLIP = 5
CLIP = [SOFT_CLIP, BARD_CLIP]

for read in sv_indicant_reads_in_contig:
    read.
    
    

IndentationError: unexpected indent (<ipython-input-24-fc61973eca9a>, line 6)

In [34]:
sv_indicant_reads_in_contig[1].get_blocks()

[(19944, 20000)]

In [39]:
def keywithmaxval(d):
     v=list(d.values())
     k=list(d.keys())
     return k[v.index(max(v))]

left_block_ends = defaultdict(lambda: 0)
right_block_ends = defaultdict(lambda: 0)
for read in sv_indicant_reads_in_contig:
    for l,r in read.get_blocks():
        left_block_ends[l] += 1
        right_block_ends[r] += 1

print 'right mode:', keywithmaxval(right_block_ends)
print 'left mode:', keywithmaxval(left_block_ends)

right mode: 20000
left mode: 20000


In [40]:
left_block_ends

defaultdict(<function <lambda> at 0x7f6f151981b8>, {19968: 1, 20139: 1, 19998: 1, 20000: 24, 20005: 1, 19509: 1, 19511: 1, 19513: 1, 19519: 1, 20033: 1, 19523: 1, 20038: 1, 20040: 1, 20042: 1, 19554: 1, 19536: 1, 19541: 1, 20056: 1, 19550: 1, 20064: 1, 19553: 1, 20066: 1, 20070: 1, 20076: 1, 20079: 2, 20080: 1, 19569: 1, 20083: 1, 19573: 1, 19576: 1, 19584: 1, 20098: 1, 20099: 1, 19564: 1, 19595: 1, 20108: 1, 20109: 1, 20333: 1, 20113: 1, 19602: 1, 19606: 1, 20120: 1, 20121: 1, 20122: 1, 19611: 1, 19612: 2, 19613: 1, 19615: 1, 19616: 1, 19568: 1, 20132: 1, 20135: 1, 19624: 1, 19627: 1, 20143: 1, 19634: 1, 19635: 1, 20148: 1, 19639: 1, 20152: 1, 20154: 2, 20155: 1, 19645: 1, 19647: 1, 20160: 1, 19650: 2, 20167: 1, 20169: 1, 20171: 1, 19660: 1, 19661: 1, 19747: 1, 19669: 1, 20191: 1, 19681: 1, 19683: 1, 20199: 1, 19692: 1, 19693: 1, 20209: 1, 19699: 1, 19700: 1, 20215: 1, 19705: 1, 19712: 1, 20239: 1, 19728: 1, 19733: 1, 19737: 1, 19738: 1, 20251: 1, 20253: 1, 20257: 1, 20259: 1, 20263: 

In [46]:
read = sv_indicant_reads_in_contig[1]
read[0:10]

TypeError: 'pysam.calignmentfile.AlignedSegment' object has no attribute '__getitem__'

In [6]:
for read in sv_indicant_reads_in_contig:
    if read.cigartuples:
        if len(read.cigartuples) > 2:
            myread = read

In [7]:
myread = sv_indicant_reads_in_contig[1]
myread.cigarstring

'56M45S'

In [8]:
MATCH = 0
INS = 1
DEL = 2
SOFT_CLIP = 4
HARD_CLIP = 5
CLIP = [SOFT_CLIP, HARD_CLIP]

for read in sv_indicant_reads_in_contig:
    read.

SyntaxError: invalid syntax (<ipython-input-8-79ec0ad27aea>, line 9)

In [9]:
read

In [24]:
MATCH = 0
INS = 1
DEL = 2
SOFT_CLIP = 4
HARD_CLIP = 5
CLIP = [SOFT_CLIP, HARD_CLIP]

def cigarLengths(cigartuples):
    alignment_length = 0
    query_start = 0
    query_end = 0
    
    is_first_op = True
    if cigartuples is not None:
        for op, op_length in cigartuples:
            if op in CLIP and is_first_op:
                query_start += op_length
                query_end += op_length
            elif op is MATCH:
                query_end += op_length
                alignment_length += op_length
            elif op is INS:
                query_end += op_length
            elif op is DEL:
                alignment_length += op_length
            is_first_op = False

    return {
        'query_start': query_start,
        'query_end': query_end,
        'alignment_length': alignment_length
    }

In [32]:
num = 7
for num in range(180,200):
    print sv_indicant_reads_in_contig[num].cigarstring
    print cigarLengths(sv_indicant_reads_in_contig[num].cigartuples)

None
{'query_start': 0, 'query_end': 0, 'alignment_length': 0}
89M12S
{'query_start': 0, 'query_end': 89, 'alignment_length': 89}
101M
{'query_start': 0, 'query_end': 101, 'alignment_length': 101}
None
{'query_start': 0, 'query_end': 0, 'alignment_length': 0}
None
{'query_start': 0, 'query_end': 0, 'alignment_length': 0}
None
{'query_start': 0, 'query_end': 0, 'alignment_length': 0}
None
{'query_start': 0, 'query_end': 0, 'alignment_length': 0}
None
{'query_start': 0, 'query_end': 0, 'alignment_length': 0}
None
{'query_start': 0, 'query_end': 0, 'alignment_length': 0}
None
{'query_start': 0, 'query_end': 0, 'alignment_length': 0}
None
{'query_start': 0, 'query_end': 0, 'alignment_length': 0}
101M
{'query_start': 0, 'query_end': 101, 'alignment_length': 101}
None
{'query_start': 0, 'query_end': 0, 'alignment_length': 0}
None
{'query_start': 0, 'query_end': 0, 'alignment_length': 0}
64S37M
{'query_start': 64, 'query_end': 101, 'alignment_length': 37}
None
{'query_start': 0, 'query_end': 

In [18]:
for read in sv_indicant_reads_in_contig:
    

{'alignment_length': 56, 'query_end': 56, 'query_start': 0}

In [36]:
sv_indicant_reads_in_contig[1].get_aligned_pairs()

[(0, 19944),
 (1, 19945),
 (2, 19946),
 (3, 19947),
 (4, 19948),
 (5, 19949),
 (6, 19950),
 (7, 19951),
 (8, 19952),
 (9, 19953),
 (10, 19954),
 (11, 19955),
 (12, 19956),
 (13, 19957),
 (14, 19958),
 (15, 19959),
 (16, 19960),
 (17, 19961),
 (18, 19962),
 (19, 19963),
 (20, 19964),
 (21, 19965),
 (22, 19966),
 (23, 19967),
 (24, 19968),
 (25, 19969),
 (26, 19970),
 (27, 19971),
 (28, 19972),
 (29, 19973),
 (30, 19974),
 (31, 19975),
 (32, 19976),
 (33, 19977),
 (34, 19978),
 (35, 19979),
 (36, 19980),
 (37, 19981),
 (38, 19982),
 (39, 19983),
 (40, 19984),
 (41, 19985),
 (42, 19986),
 (43, 19987),
 (44, 19988),
 (45, 19989),
 (46, 19990),
 (47, 19991),
 (48, 19992),
 (49, 19993),
 (50, 19994),
 (51, 19995),
 (52, 19996),
 (53, 19997),
 (54, 19998),
 (55, 19999),
 (56, None),
 (57, None),
 (58, None),
 (59, None),
 (60, None),
 (61, None),
 (62, None),
 (63, None),
 (64, None),
 (65, None),
 (66, None),
 (67, None),
 (68, None),
 (69, None),
 (70, None),
 (71, None),
 (72, None),
 (73,

In [50]:
SOFT_CLIP = 4
HARD_CLIP = 5
CLIP = [SOFT_CLIP, HARD_CLIP]

left_clipped = defaultdict(list)
right_clipped = defaultdict(list)
for read in sv_indicant_reads_in_contig:
    left_clipping = read.cigartuples[0][1] if read.cigartuples[0][0] in CLIP else 0
    right_clipping = read.cigartuples[-1][1] if read.cigartuples[-1][0] in CLIP else 0
    is_left_clipped = left_clipping > right_clipping
    is_right_clipped = right_clipping > left_clipping
    if is_left_clipped:
        left_clipped[read.reference_start].append(read)
    elif is_right_clipped:
        right_clipped[read.reference_end].append(read)



In [158]:
SOFT_CLIP = 4
HARD_CLIP = 5
CLIP = [SOFT_CLIP, HARD_CLIP]

# Separate left and right clipped reads
left_clipped = defaultdict(list)
right_clipped = defaultdict(list)
for read in sv_indicant_reads_in_contig:
    left_clipping = read.cigartuples[0][1] if read.cigartuples[0][0] in CLIP else 0
    right_clipping = read.cigartuples[-1][1] if read.cigartuples[-1][0] in CLIP else 0
    is_left_clipped = left_clipping > right_clipping
    is_right_clipped = right_clipping > left_clipping
    if is_left_clipped:
        left_clipped[read.reference_start].append(read)
    elif is_right_clipped:
        right_clipped[read.reference_end].append(read)

# Find positions in reference of most left clipping points
left_clipped_list_sorted = sorted(left_clipped.items(), key=lambda x:len(x[1]), reverse=True)
highest_clip_consensus = len(left_clipped_list_sorted[0][1])
second_highest_clip_consensus = len(left_clipped_list_sorted[1][1]) if len(left_clipped_list_sorted) > 1 else 0
if highest_clip_consensus - second_highest_clip_consensus > 2:
    ref_ins_right_end = left_clipped_list_sorted[0][0]
else:
    ref_ins_right_end = None

# Same for right clipping
right_clipped_list_sorted = sorted(right_clipped.items(), key=lambda x:len(x[1]), reverse=True)
highest_clip_consensus = len(right_clipped_list_sorted[0][1])
second_highest_clip_consensus = len(right_clipped_list_sorted[1][1]) if len(right_clipped_list_sorted) > 1 else 0
if highest_clip_consensus - second_highest_clip_consensus > 2:
    ref_ins_left_end = right_clipped_list_sorted[0][0]
else:
    ref_ins_left_end = None

# TODO: Handle case of no endpoints found
assert ref_ins_left_end is not None and ref_ins_right_end is not None
print 'ref_ins_left_end:', ref_ins_left_end
print 'ref_ins_right_end:', ref_ins_right_end


# Grab query_alignment_sequences for alignment to contig

# TODO: Handle case of same query_name reads being added to same fastq
right_clipped_query_names = [read.query_name for read in right_clipped[ref_ins_left_end]]
assert len(right_clipped_query_names) == len(set(right_clipped_query_names))

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
from Bio import SeqIO
right_clipped_query_alignment_seqrecords = []
for read in right_clipped[ref_ins_left_end]:
    right_clipped_query_alignment_seqrecords.append(SeqRecord(
            Seq(read.query_alignment_sequence, IUPAC.ambiguous_dna),
            letter_annotations={'phred_quality':read.query_alignment_qualities},
            id=read.query_name,
            description=''))

right_clipped_query_alignment_fastq = os.path.join(read_unpacking_dir, 'right_clipped_query_alignment_seqs.fq')
with open(right_clipped_query_alignment_fastq, 'w') as fastq_handle:
    SeqIO.write(right_clipped_query_alignment_seqrecords, fastq_handle, 'fastq')


input_reads_fq = right_clipped_query_alignment_fastq
from main.models import Contig
from main.models import Dataset
contig = Contig.objects.get(label='ins_1kb_ins_1kb_sample_NODE_1_length_1966_cov_23.051373')
contig_fasta = contig.dataset_set.get(type=Dataset.TYPE.REFERENCE_GENOME_FASTA).get_absolute_location()

# Align clipped query alignment fastq to contig
align_input_args = ' '.join([
        '%s/bwa/bwa' % settings.TOOLS_DIR,
        'mem',
        '-t', '1', # threads
        contig_fasta,
        input_reads_fq,
    ])

# To skip saving the SAM file to disk directly, pipe output directly to
# make a BAM file.
align_input_args += ' | ' + settings.SAMTOOLS_BINARY + ' view -bS -'

# Get a BAM filename
output_bam = os.path.join(read_unpacking_dir,
        'right_clipped_to_contig.bwa_align.bam')

# Ensure contig fasta is indexed
subprocess.call(' '.join([
        '%s/bwa/bwa' % settings.TOOLS_DIR,
        'index',
        contig_fasta
    ]),
shell=True, executable=settings.BASH_PATH)

# Run alignment
import subprocess
with open(output_bam, 'w') as fh:
    subprocess.check_call(
            align_input_args, stdout=fh,
            shell=True, executable=settings.BASH_PATH)

# # Check if reverse complement contig
# right_clipped_qname_to_read = {}
# for read in right_clipped[ref_ins_left_end]:
#     right_clipped_qname_to_read[read.query_name] = read

total_mapped_count = 0
reversed_complementarity_count = 0
sam_file = pysam.AlignmentFile(output_bam)
for read in sam_file:
    if not read.is_unmapped:
        total_mapped_count += 1
        if read.is_reverse:
            reversed_complementarity_count += 1

REVERSED_COMPLEMENTARITY_FRACTION_CUTOFF = 0.75
if reversed_complementarity_count / total_mapped_count > REVERSED_COMPLEMENTARITY_FRACTION_CUTOFF:
    print 'Contig is reverse complement'
    contig.metadata['is_reverse'] = True

# Write reverse complement of contig to file if is reverse
if contig.metadata.get('is_reverse', False):
    rc_contig_fasta = os.path.splitext(contig_fasta)[0] + '.reverse_complement.fa'
    print 'reverse complement contig fasta:', rc_contig_fasta
    contig_seqrecord = SeqIO.parse(contig_fasta, 'fasta').next()
    contig_seqrecord.seq = contig_seqrecord.seq.reverse_complement()
    SeqIO.write(contig_seqrecord, rc_contig_fasta, 'fasta')
    
    # Align clipped query alignment fastq to contig
    align_input_args = ' '.join([
            '%s/bwa/bwa' % settings.TOOLS_DIR,
            'mem',
            '-t', '1', # threads
            rc_contig_fasta,
            input_reads_fq,
        ])

    # To skip saving the SAM file to disk directly, pipe output directly to
    # make a BAM file.
    align_input_args += ' | ' + settings.SAMTOOLS_BINARY + ' view -bS -'

    # Get a BAM filename
    output_bam = os.path.join(read_unpacking_dir,
            'right_clipped_to_contig.bwa_align.bam')

    # Ensure contig fasta is indexed
    subprocess.call(' '.join([
            '%s/bwa/bwa' % settings.TOOLS_DIR,
            'index',
            rc_contig_fasta
        ]),
    shell=True, executable=settings.BASH_PATH)

    # Run alignment
    import subprocess
    with open(output_bam, 'w') as fh:
        subprocess.check_call(
                align_input_args, stdout=fh,
                shell=True, executable=settings.BASH_PATH)


alignment_ref_end_positions = defaultdict(list)
sam_file = pysam.AlignmentFile(output_bam)
for read in sam_file:
    # Change to reference_start for left_clipped
    alignment_ref_end_positions[read.reference_end].append(read)
#     print 'read.reference_start:', read.reference_start
#     print 'read.reference_end:', read.reference_end

alignment_ref_end_positions_sorted = sorted(alignment_ref_end_positions.items(), key=lambda x:len(x[1]), reverse=True)
highest_end_consensus = len(alignment_ref_end_positions_sorted[0][1])
second_highest_end_consensus = len(alignment_ref_end_positions_sorted[1][1]) if len(alignment_ref_end_positions_sorted) > 1 else 0
if highest_end_consensus - second_highest_end_consensus > 2:
    contig_ins_left_end = alignment_ref_end_positions_sorted[0][0]
else:
    contig_ins_left_end = None

print 'contig_ins_left_end:', contig_ins_left_end


ref_ins_left_end: 20000
ref_ins_right_end: 20000
Contig is reverse complement
reverse complement contig fasta: /home/wahern/projects/millstone/genome_designer/conf/../temp_data/projects/ecbcef23/contigs/901e5a0d/fasta.reverse_complement.fa
contig_ins_left_end: 491


In [156]:
# sv_indicant_reads_in_contig[1].mapping_quality
# print right_clipped_query_alignment_seqs[0]
# right_clipped_query_alignment_fastq
# myread.mapping_quality
# 10**(-0.1*12)
# -10*np.log10(0.1)
# alignment_ref_end_positions.items()
# second_highest_end_consensus
# right_clipped[20000][2].reference_start
# os.path.splitext(contig_fasta)
contig.metadata

{u'coverage': 23.051373, u'timestamp': u'2015-07-15 11:46:25.414577'}

In [66]:
from Bio import SeqIO

In [78]:
#seqrec = SeqIO.SeqRecord('AGAGT', 'phred_quality'=[24,24,21,32,38])

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
record = SeqRecord(Seq("ATGG",
                   IUPAC.ambiguous_dna),
                   letter_annotations={'phred_quality':[32,8,23,25]},
                   id="YP_025292.1", name="HokC",
                   description="toxic membrane protein, small")
print record

ID: YP_025292.1
Name: HokC
Description: toxic membrane protein, small
Number of features: 0
Per letter annotation for: phred_quality
Seq('ATGG', IUPACAmbiguousDNA())
